### Setting local .ddl file

In [30]:
#https://silentinstallhq.com/microsoft-analysis-services-adomd-net-silent-install-how-to-guide/
import clr

path_ddl = 'C:/Program Files (x86)/Microsoft.NET/ADOMD.NET/160/Microsoft.AnalysisServices.AdomdClient.dll'

clr.AddReference(path_ddl)
clr.AddReference('Microsoft.AnalysisServices.AdomdClient')

In [28]:
# List of substrings to check for in the paths
substrings_to_remove = ["ADOMD.NET"]

# Filter the paths and create a new list without the unwanted substrings
new_path = [p for p in sys.path if all(sub not in p for sub in substrings_to_remove)]

# Update sys.path to use the new list
sys.path = new_path

### Setting Path

In [9]:
import pandas as pd
from sys import path
import sys
import os

path.append(path_ddl)
print(path)

from pyadomd import Pyadomd

['C:\\Users\\ADMIN\\Desktop\\TDW_SCRIPTS', 'C:\\Users\\ADMIN\\anaconda3\\python39.zip', 'C:\\Users\\ADMIN\\anaconda3\\DLLs', 'C:\\Users\\ADMIN\\anaconda3\\lib', 'C:\\Users\\ADMIN\\anaconda3', '', 'C:\\Users\\ADMIN\\AppData\\Roaming\\Python\\Python39\\site-packages', 'C:\\Users\\ADMIN\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32', 'C:\\Users\\ADMIN\\AppData\\Roaming\\Python\\Python39\\site-packages\\win32\\lib', 'C:\\Users\\ADMIN\\AppData\\Roaming\\Python\\Python39\\site-packages\\Pythonwin', 'C:\\Users\\ADMIN\\anaconda3\\lib\\site-packages', 'C:\\Users\\ADMIN\\anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\ADMIN\\anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\ADMIN\\anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\ADMIN\\anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\ADMIN\\.ipython', 'C:\\Windows\\Microsoft.NET\\Framework64\\v4.0.30319\\', 'C:/Program Files (x86)/Microsoft.NET/ADOMD.NET/160/Microsoft.AnalysisServices.AdomdClient.dll']


### Connection to Database from PBI Report and getting data

In [10]:
# Database ID and port from dax studio in select * from $SYSTEM.DBSCHEMA_CATALOGS
model_name = '58e307a4-fc93-43d0-a464-a74463cba719'
port_number = 'localhost:20408'

conn_string = f'Provider=MSOLAP;Data Source={port_number};Catalog={model_name};'
conn = Pyadomd(conn_string)


def get_PBIData(dax_query,conn):
    with conn as con:
        with con.cursor().execute(dax_query) as cur:
            DF = pd.DataFrame(cur.fetchone(), columns = [i.name for i in cur.description])
    
    return DF

In [26]:
# Getting data
dax_query = '''
EVALUATE 
SUMMARIZECOLUMNS(
Fecha[Date],
Productos[proId],
Productos[proNombre],
Productos[Categoria],
Productos[Subcategoria],
Tiendas[sucId],
Tiendas[NombreSucursal],
FILTER(Fecha,Fecha[Año] >=2022),
"VENTA", [Venta]
)
'''

df_ventas = get_PBIData(dax_query,conn)
df_ventas.head()

,Fecha[Date],Productos[proId],Productos[proNombre],Productos[Categoria],Productos[Subcategoria],Tiendas[sucId],Tiendas[NombreSucursal],[VENTA]
0,2023-09-15,29622.0,CAFE AMERICANO AE-02,Otras bebidas,No alcohol,8,AEROPUERTO- 2,77.44
1,2023-09-14,29622.0,CAFE AMERICANO AE-02,Otras bebidas,No alcohol,8,AEROPUERTO- 2,89.85
2,2023-09-13,29622.0,CAFE AMERICANO AE-02,Otras bebidas,No alcohol,8,AEROPUERTO- 2,102.23
3,2023-09-03,29622.0,CAFE AMERICANO AE-02,Otras bebidas,No alcohol,8,AEROPUERTO- 2,55.77
4,2023-08-20,29622.0,CAFE AMERICANO AE-02,Otras bebidas,No alcohol,8,AEROPUERTO- 2,49.59


In [29]:
df_ventas.shape

(589661, 8)